# Example Generic Sim Sweep

In this example, we will demonstrate how to run a generic sim sweep for EMOD.

First, we need to import the necessary modules classes.

In [ ]:
import os
from functools import partial

from config_update_parameters import config_update_params

from idmtools.builders import ExperimentBuilder, StandAloneSimulationsBuilder
from idmtools.core import ItemType
from idmtools.core.platform_factory import Platform
from idmtools.managers import ExperimentManager
from idmtools_model_emod import EMODExperiment
from idmtools_model_emod.defaults import EMODSir
from idmtools_model_emod.generic.serialization import add_serialization_timesteps, load_serialized_population
from idmtools_test import COMMON_INPUT_PATH
from idmtools_test.utils.comps import sims_from_experiment, get_simulation_path

And set some paths.

In [ ]:
current_directory = os.path.dirname(os.path.realpath(__file__))
BIN_PATH = os.path.join(current_directory, "bin")
INPUT_PATH = os.path.join(current_directory, "inputs")

sim_duration = 10  # in years
num_seeds = 5

Update our parameters.

In [ ]:
def param_update(simulation, param, value):
    return simulation.set_parameter(param, value)


In order to run the experiment, we need to create a Platform and an ExperimentManager. The Platform defines where we want to run our simulation. LocalPlatform used here will run the simulations on your local machine.

In [ ]:
platform = Platform('COMPS')

In [ ]:
sim_duration = 2  # in years
num_seeds = 1

expname = 'Generic EMOD experiment with param sweeps'
e1 = EMODExperiment.from_default(expname, default=EMODSir(),
                                     eradication_path=os.path.join(BIN_PATH, "Eradication.exe"))

simulation = e1.base_simulation

Update config parameters that you need to update:

In [ ]:
sim = config_update_params(simulation)
timesteps = [sim_duration * 365]
add_serialization_timesteps(simulation=sim, timesteps=[sim_duration * 365],
                                end_at_final=False, use_absolute_times=False)

start_day = sim.get_parameter("Start_Time")
last_serialization_day = sorted(timesteps)[-1]
end_day = start_day + last_serialization_day
sim.set_parameter("Simulation_Duration", end_day)

Now add your sweeps:

In [ ]:
# Sweep parameters
builder = ExperimentBuilder()
set_Run_Number = partial(param_update, param="Run_Number")
builder.add_sweep_definition(set_Run_Number, range(num_seeds))
e1.tags = {'idmtools': 'generic EMOD experiment with param sweeps'}
set_x_Temporary_Larval_Habitat = partial(param_update, param="x_Temporary_Larval_Habitat")
builder.add_sweep_definition(set_x_Temporary_Larval_Habitat, [0.1, 0.2])

Add simulation tags:

In [ ]:
# Add simulation tags
e1.base_simulation.tags = {'add_base_sim_tag': 'my_tag'}
# another way to add custom simulation tags with add_sweep_definition
set_tag = partial(param_update, param="test_tag")
builder.add_sweep_definition(set_tag, "abcd")

The last step is to call `run()` on the `ExperimentManager` to run the simulations.

In [ ]:
    e1.builder = builder
    em = ExperimentManager(experiment=e1, platform=platform)
    em.run()
    em.wait_till_done()